# Janssen-Cilag - List
## Beachten
Kein PDF verfügbar. Werte wurden von Hand von der Website kopiert (jeweils mit Kopfzeile) und in einem Texteditor (nicht Excel!) abgespeichert.
* https://public.janssentransferofvalue.com/ch_de/hcp-individual
* https://public.janssentransferofvalue.com/ch_de/hco-individual

In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

## Import PDF

In [2]:
# HCP
df_hcp = pd.read_csv("hcp.csv", sep="\t", skiprows=2, names=['_nachname', '_vorname', 'location', 'address', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses'])
df_hcp['type'] = 'hcp'

#HCO
df_hco = pd.read_csv("hco.csv", sep="\t", skiprows=2, names=['name', 'location', 'address', 'donations_grants', 'sponsorship', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses'])
df_hco['type'] = 'hco'

#Concat
df = pd.concat([df_hcp, df_hco], sort=False)

## Format Table

In [3]:
df_export = df.copy()

#Rename Columns
#df_export.columns = ['_nachname', '_vorname', 'location', 'address', 'registration_fees', 'travel_accommodation', 'fees', 'related_expenses']

#Remove where _nachname = "NACHNAME" or "Angaben"
df_export = df_export[df_export['_nachname'] != 'NACHNAME']
df_export = df_export[df_export['_nachname'] != 'Angaben']
df_export = df_export[df_export['name'] != 'VOLLSTÄNDIGER NAME']
df_export = df_export[df_export['name'] != 'Angaben']

#Merge _nachname _vorname and remove fields
#df_export.loc[df_export['_vorname'].notna(), 'name'] = df_export['_vorname'] + ' ' + df_export['_nachname']
df_export['_name'] = df_export['_vorname'] + ' ' + df_export['_nachname']
df_export.loc[df_export['name'].isna(), 'name'] = df_export.loc[df_export['name'].isna(), '_name']
df_export.drop(columns=['_vorname', '_nachname', '_name'], inplace=True)

#Add missed rows
df_export.insert(0, 'total', "")
df_export.insert(0, 'country', "CH")
df_export.insert(0, 'plz', "")
df_export.insert(0, 'uci', "")

#Convert to Numbers
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)
df_export = sum_amounts(df_export)

#basic string conversion
df_export = basic_string_conversion(df_export)

#Reorder columns
df_export = df_export[fix_columns[:-1]]

#Export
export_list(df_export, 'janssen')

saved


In [4]:
#write_to_csv(df_export, 'tmp.csv')
#write_to_excel(df_export, 'tmp.xlsx', open=True)